In [2]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 62 kB/s 
     |████████████████████████████████| 198 kB 41.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=c7d6dd66c0e6c12679ce3d1d65414d4ca9d9361b898828a60333e1c9120541e5
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [3]:
from pyspark.sql.types import *
from operator import add
from pyspark.sql import functions as F
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
import string
from pyspark.ml.feature import StopWordsRemover
from pyspark.sql.window import Window
from gensim.parsing.preprocessing import STOPWORDS

conf = SparkConf().setMaster("local").setAppName("My App")
sc = SparkContext(conf = conf)
spark = SparkSession(sc)

stop_words = list(STOPWORDS)
stop_words.extend(list(''+' '+string.punctuation + string.digits))

with open("/content/nytimes_news_articles.txt", "r") as f:  # 打开文件
    data = f.read()  # 读取文件
data = data.split('\n\n')
key = []
for i in range(len(data)):
    if 'URL: http://www.nytimes.com/' in data[i]:
        tem = '/'.join(data[i][39:].split('/')[:-1])
        continue
    temp = data[i].lower()
    key.append((tem, temp.split(' ')))

pairRDD = sc.parallelize(key).toDF(['class','message'])
remover = StopWordsRemover(stopWords=stop_words,inputCol='message', outputCol='message_clean')
pairRDD = remover.transform(pairRDD).select('class', 'message_clean')
count_RDD=pairRDD.withColumn("word",F.explode("message_clean")).groupBy("class","word").count()
windows_spec = Window.partitionBy("class").orderBy(F.col("count").desc())
most_words = count_RDD.withColumn("rank", F.rank().over(windows_spec)).filter("rank <=10")
most_words.show()

most_words.toPandas().to_csv('/content/task3_ans.csv')

+-------------------+---------+-----+----+
|              class|     word|count|rank|
+-------------------+---------+-----+----+
|             travel|        —|  610|   1|
|             travel|     like|  352|   2|
|             travel|      new|  313|   3|
|             travel|     said|  218|   4|
|             travel|     it’s|  206|   5|
|             travel|    hotel|  202|   6|
|             travel|   travel|  187|   7|
|             travel|   people|  155|   8|
|             travel|      mr.|  150|   9|
|             travel|including|  149|  10|
|             travel|     city|  149|  10|
|sports/horse-racing|    derby|  146|   1|
|sports/horse-racing|    horse|  122|   2|
|sports/horse-racing|        —|   90|   3|
|sports/horse-racing| kentucky|   88|   4|
|sports/horse-racing|     said|   85|   5|
|sports/horse-racing|     colt|   84|   6|
|sports/horse-racing|   horses|   81|   7|
|sports/horse-racing|  nyquist|   76|   8|
|sports/horse-racing|    said.|   72|   9|
+----------